# Acesso ao SD

## Configuração do ambiente

### Na máquina local

Pré-requisito: distribuição Python (Anaconda ou Intel) instalada e funcionando na máquina local

* https://www.anaconda.com/products/individual
* https://software.intel.com/content/www/us/en/develop/tools/oneapi/components/distribution-for-python.html


### Exemplo de instalação do Jupyter-Lab

Distribuição Intel (que inclui outros compiladores e ferramentas Intel)

* https://software.intel.com/content/www/us/en/develop/documentation/installation-guide-for-intel-oneapi-toolkits-linux/top/installation/install-using-package-managers/apt.html

Usando Ubuntu 20.04

```
...
sudo apt install intel-basekit intel-hpckit
sudo chmod -R 777 /opt/intel/oneapi/intelpython
source /opt/intel/oneapi/setvars.sh
conda install jupyterlab
jupyter-lab
```


### Exemplos de configuração do VPN, SSH, e JupyterLab

* https://github.com/efurlanm/hpc/blob/master/sd/access_configuring.ipynb
* https://www.hostinger.com.br/tutoriais/conexao-ssh-sem-senha
* https://github.com/efurlanm/hpc/blob/master/sd/access_using_jupyterlab.ipynb

> Nota: o SSH foi configurado para chave ao invés de senha, tanto na máquina local quanto no SD para acessar o sdumont18

Configuração deste Notebook:

In [2]:
bind 'set enable-bracketed-paste off'

## Acessando o Santos Dumont

* Primeiro passo: usando o Gerenciador de Rede, fazer a conexão ao VPN Cisco usando vpnc

Ao fazer login normalmente, uma máquina de login é selecionada aleatoriamente (geralmente sdumont11, sdumont13, ou sdumont14). O comando a seguir abre uma conexão mestre ("SSH Socket") em um dos nós, de tal forma que todos os comandos subsequentes sejam realizados neste mesmo nó:

In [2]:
export SSH_SOCKET=~/.ssh/remote_sd

In [3]:
ssh -MNfo ControlPath=$SSH_SOCKET sd    # não esquecer do VPN

Mostra em qual nó estamos conectados:

In [4]:
ssh -S $SSH_SOCKET sd 'hostname'

sdumont14


## Primeira sessão, no nó B-710

O comando `ssh` a seguir, executa os seguintes comandos no SD:

* carrega o módulo anaconda
* carraga uma instância do servidor JupyterLab, que fica ativo na porta 20210 (pode escolher outra porta)

Uma instância JupyterLab fica rodando na máquina de login onde a conexão mestre está apontando

In [5]:
ssh -S $SSH_SOCKET sd 'module load anaconda3; jupyter-lab --no-browser --port=20210 --ip=0.0.0.0 > ~/20210.log 2>&1 & '

## Segunda sessão, no nó sequana (sdumont18)

* entra no nó Sequana sdumont18 usando o comando ssh que é executado na máquina de login
* carrega uma nova instância do servidor JupyterLab, que fica ativo no nó sdumont18 na porta 20211
* o ideal é usar portas aleatórias que ninguém mais esteja usando

In [6]:
ssh -S $SSH_SOCKET sd 'ssh -i ~/.ssh/id_sd_ed25519 sdumont18 "module load anaconda3; jupyter-lab --no-browser --port=20211 --ip=0.0.0.0 > ~/20211.log 2>&1 &" '

## Conexão das instâncias Jupyter à máquina local

Usando encaminhamento de portas. Localmente o acesso é feito usando as portas 8889 e 8890 (pode escolher outras portas)

In [7]:
ssh -S $SSH_SOCKET sd -NTfL 8889:localhost:20210

In [8]:
ssh -S $SSH_SOCKET sd -NTfL 8890:sdumont18:20211

## Acesso aos arquivos usando o Gerenciador de Arquivos

* É independente do Jupyter, pode ser usado apenas o Gerenciador
* É importante criar os diretórios /pr e /scratch na máquina local, para evitar paths ou links quebrados.

In [1]:
sshfs sd:/prj     /prj     -o workaround=rename,uid=1000,gid=1000
sshfs sd:/scratch /scratch -o workaround=rename,uid=1000,gid=1000

: 1

## Links para os clientes JupyterLab

* se não aparecer nada ou aparecer em branco, tem que esperar um pouco para o servidor subir
* se assim mesmo o link não aparecer, ou der erro: mata o processo e sobe de novo

In [10]:
ssh -S $SSH_SOCKET sd 'cat ~/20210.log | grep "     or" ' > ~/tk1.txt
temp=$(cat ~/tk1.txt)
echo ${temp/#*'20210'/'http://localhost:8889'}

http://localhost:8889/?token=7b821a4a74023d35e1f0c49abd50af077b538a888d8e4d61


In [11]:
ssh -S $SSH_SOCKET sd 'cat ~/20211.log | grep "     or" ' > ~/tk2.txt
temp=$(cat ~/tk2.txt)
echo ${temp/#*'20211'/'http://localhost:8890'}

http://localhost:8890/?token=2da8b0e447b136d02fb1bca015b57e3d71fbc09fb260bb68


# Sair

Uma forma é usando o menu do cliente JupyterLab e selecionando "File > Shut Down" para cada instância do servidor, e em seguida encerrar a conexão mestre:

In [4]:
ssh -S $SSH_SOCKET sd -O exit

Exit request sent.


Outra forma é matando os processos:

In [14]:
ssh -S $SSH_SOCKET sd 'pkill -u ${USER} -f jupyter-lab'

: 1

In [15]:
ssh -S $SSH_SOCKET sd 'ssh -i ~/.ssh/id_sd_ed25519 sdumont18 "pkill -u ${USER} -f jupyter-lab" '

: 1

In [16]:
ssh -S $SSH_SOCKET sd -O exit

Exit request sent.


### Desmonta filesystem remoto, e encerra encaminhamento de portas:

In [17]:
sudo umount -fl /prj /scratch

In [18]:
pkill -f 8889:

: 1

In [19]:
pkill -f 8890:

: 1